In [1]:
import json
import requests
from datetime import datetime, timedelta
import pickle
import xgboost
import pandas as pd

In [2]:
def get_forecast_hourly_weather():
        url = "https://api.open-meteo.com/v1/forecast?latitude=50.88&longitude=4.70&hourly=temperature_2m,relativehumidity_2m,dewpoint_2m,apparent_temperature,rain,showers,snowfall,weathercode,pressure_msl,surface_pressure,cloudcover,cloudcover_low,cloudcover_mid,cloudcover_high,evapotranspiration,et0_fao_evapotranspiration,vapor_pressure_deficit,windspeed_10m,winddirection_10m,windgusts_10m,shortwave_radiation,direct_radiation,diffuse_radiation,direct_normal_irradiance&models=best_match&timezone=Europe%2FBerlin"
        resp = requests.get(url)
        data = resp.json()
        df = pd.DataFrame(data['hourly'])
        return df

def get_forecast_hourly_air():
        url = "https://air-quality-api.open-meteo.com/v1/air-quality?latitude=50.88&longitude=4.70&hourly=pm10,pm2_5,carbon_monoxide,nitrogen_dioxide,sulphur_dioxide,ozone,ammonia&timezone=Europe%2FBerlin"
        resp = requests.get(url)
        data = resp.json()
        df = pd.DataFrame(data['hourly'])
        return df

In [4]:
# Get 4 days 
today = datetime.now().date()
next_4_days = [today + timedelta(days=i) for i in range(4)]

In [6]:
# Get weather
hourly_weather = get_forecast_hourly_weather()
hourly_weather["time"] = pd.to_datetime(hourly_weather["time"])
hourly_weather = hourly_weather[hourly_weather["time"].dt.date.isin(next_4_days)]
#selected_hourly_weather = hourly_weather[hourly_weather["time"].dt.date == selected_date]
hourly_weather['precipitation'] = hourly_weather['rain'] + hourly_weather['showers'] + hourly_weather['snowfall']
hourly_weather.drop(['showers', 'et0_fao_evapotranspiration', 'evapotranspiration', 'vapor_pressure_deficit'], axis=1, inplace=True)

# Get air
hourly_air = get_forecast_hourly_air()
hourly_air["time"] = pd.to_datetime(hourly_air["time"])
hourly_air = hourly_air[hourly_air["time"].dt.date.isin(next_4_days)]
#selected_hourly_air = hourly_air[hourly_air["time"].dt.date == selected_date]
hourly_air.rename(columns={'carbon_monoxide':'co', 
                                    'nitrogen_dioxide':'no2', 
                                    'sulphur_dioxide':'so2', 
                                    'ozone':'o3',
                                    'ammonia':'nh3'}, inplace=True)


# Merge
hourly_df = pd.merge(hourly_weather, hourly_air, on='time')
hourly_df['date'] = hourly_df['time'].dt.date
hourly_df['hour'] = hourly_df['time'].dt.hour
hourly_df["month"] = hourly_df["time"].dt.month
hourly_df['weekday'] = hourly_df['time'].dt.strftime('%a')
hourly_df.drop(['time'], axis=1, inplace=True)
hourly_df.head(3)

,temperature_2m,relativehumidity_2m,dewpoint_2m,apparent_temperature,rain,snowfall,weathercode,pressure_msl,surface_pressure,cloudcover,...,pm2_5,co,no2,so2,o3,nh3,date,hour,month,weekday
0,11.4,73,6.8,9.6,0.0,0.0,0,1024.7,1021.5,0,...,13.8,162.0,18.4,1.6,58.0,3.4,2023-05-25,0,5,Thu
1,10.4,79,6.9,8.7,0.0,0.0,0,1025.3,1022.1,2,...,13.0,158.0,15.3,1.2,54.0,3.9,2023-05-25,1,5,Thu
2,9.4,85,7.0,7.7,0.0,0.0,0,1025.3,1022.1,0,...,15.3,151.0,11.2,0.8,59.0,4.4,2023-05-25,2,5,Thu


In [ ]:
with open('model_test_v1.pkl', 'rb') as file:
    model_v1 = pickle.load(file)

In [ ]:
predictions = model_v1.predict(hourly_df)